In [1]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Embedding, Flatten, Conv1D, MaxPooling1D, Attention, Permute, Reshape, Lambda, RepeatVector, multiply, Bidirectional
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from keras import backend as K

Using TensorFlow backend.


In [4]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.00001)

max_words = 1000
max_len = 220

######################################
## Model Options ##
######################################
# options: lstm, cnn, lstm-attn, bilstm
MODEL_NAME = 'bilstm'

In [6]:
def create_lstm():
    Inp = Input(name='inputs', shape=[max_len])
    x = Embedding(max_words, 50, input_length=max_len)(Inp)
    x = LSTM(64, name='LSTM_01')(x)
    x = Dropout(0.5, name='Dropout')(x)
    x = Dense(128, activation='relu',name='Dense_01')(x)
    # x = Dropout(0.5,name='Dropout')(x)
    out = Dense(1,activation='sigmoid', name='output')(x)
    model = Model(inputs=Inp, outputs=out)
    
    return model

In [7]:
def create_cnn():
    model_CNN = Sequential(name="CNN_with_embeddings")
    model_CNN.add(Embedding(max_words, 50, input_length=max_len))
    model_CNN.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
    model_CNN.add(MaxPooling1D(pool_size=2))
    model_CNN.add(Flatten())
    model_CNN.add(Dropout(0.5))
    model_CNN.add(Dense(10, activation='relu'))
    model_CNN.add(Dense(1, activation='sigmoid'))
    
    return model_CNN

In [5]:
def create_bilstm():
    Inp = Input(name='inputs',shape=[max_len])
    x = Embedding(max_words,50,input_length=max_len)(Inp)
    x = Bidirectional(LSTM(64))(x)
    x = Dropout(0.5,name='Dropout')(x)
    x = Dense(128,activation='relu',name='Dense_01')(x)
    out = Dense(1,activation='sigmoid', name='output')(x)
    model = Model(inputs=Inp,outputs=out)

    return model

In [5]:
SINGLE_ATTENTION_VECTOR = False
APPLY_ATTENTION_BEFORE_LSTM = False
INPUT_DIM = 50
TIME_STEPS = max_len

def attention_3d_block(inputs):
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) 
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

def create_lstm_attn():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    lstm_units = 64
    lstm_out = LSTM(lstm_units, return_sequences=True)(layer)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
    model = Model([inputs], output)
    return model

In [8]:
def create_nn_model():
    global MODEL_NAME
    MODEL_NAME = MODEL_NAME.lower()
    if MODEL_NAME == 'lstm':
        model = create_lstm()
        return model
    
    elif MODEL_NAME == 'cnn':
        model = create_cnn()
        return model
    
    elif MODEL_NAME == 'lstm-attn':
        model = create_lstm_attn()
        return model
    
    elif MODEL_NAME == 'bilstm':
        model = create_bilstm()
        return model

In [9]:
model = create_nn_model()

In [8]:
def create_joint_model(df):
    inputB = Input(shape=(df.shape[1],))
    c = Dense(2, activation='relu')(inputB)
    c = Dense(4, activation='relu')(c)
    c = Dense(1, activation='sigmoid')(c)
    full_model = Model(inputs=inputB, outputs=c)

    full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                          metrics=['acc', f1_m, precision_m, recall_m])
    
    return full_model

In [9]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)